In [90]:
import ssl
context = ssl._create_unverified_context()

In [91]:
#import essential libraries
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as ureq
import logging
import pandas as pd

In [92]:
#Provided the Base URL
rest_url = "https://www.kayak.com/London.28501.restaurantlist.ksp"
rest_url

'https://www.kayak.com/London.28501.restaurantlist.ksp'

In [93]:
#request and read URL
url_client = ureq(rest_url,context=context)
rest_card = url_client.read()
#Parse and Clean HTML
rest_html = bs(rest_card,'html.parser')

In [80]:
#Parsed a big box to get url of sub pages, also indexed it upto 500
big_box = rest_html.findAll("li", {"class":"col _icz _iha _iaS _id7 _iaU _irF _iaV _irG _FE _jSU _iad _iae _qV _h-- _iaa _iaW _iaX _icu _icv _icw _icx _ics _y1 _ict _h-8 _ize _ial _ii0 _ii2 _iir _iaj"})
big_box = big_box[0:500]

In [88]:
#Created an empty list to store the data
rest_data = []
#run a Loop to open the links from the first page
for i in big_box:
  rest_link="https://www.kayak.com"+i.a['href']
  rest_request = requests.get(rest_link)
  rest_details = bs(rest_request.text,'html.parser')
  #Fetched the name of restaurants
  rest_name = rest_details.findAll("h1",{"class":"_B4 _FI _FL _FK _igH _ial _idA _iQh _iaj"})[0].text
  #created a detail box to itterate the data from same class
  detail_box = rest_details.findAll("td",{"class":"head _h-2 _ial _jgc _ij1 _iQh _iaj _Ap"})
  for j in detail_box:
    if j.text == "Cuisines":
       cusine = j.find_next_sibling("td").text
    if j.text == "Phone number":
      phone = j.find_next_sibling("td").text
    if j.text == "Website":
      website = j.find_next_sibling("td").a['href']
    if j.text =="Location":
      location= j.find_next_sibling ("td").text
    if j.text =="Payment options":
      payment_option = j.find_next_sibling("td").text
    if j.text == "Dining style":
      d_style = j.find_next_sibling("td").text
      #stored the data in dictionary
    data = {"Name":rest_name,
          "Cusines":cusine,
          "Dining_style":d_style,
          "Phone number":phone,
          "Website": website,
          "Location":location,
          "Payment Options":payment_option,    
    }
    #appended the data in list
  rest_data.append(data)

In [89]:
#convert dictionary into Dataframe and extracted the CSV
df = pd.DataFrame(rest_data)
df.to_csv("rest_data.csv")